# מדריך מקיף: בדיקות וניפוי שגיאות ב-Polars

## 🎯 מטרות המדריך

במדריך זה תלמדו:
- כיצד לנפות שגיאות בפעולות משורשרות (chained operations)
- איך לבדוק ולאפטם את תוכנית השאילתה (query plan)
- כיצד להשתמש בספרייה `cuallee` לבדיקת איכות נתונים
- יסודות בדיקות אוטומטיות עם Pytest

---

## 📚 תוכן עניינים

1. [הגדרות ראשוניות](#setup)
2. [ניפוי שגיאות בפעולות משורשרות](#debugging-chained)
   - [הבעיה: שרשראות ארוכות](#problem)
   - [פתרון 1: הערת קוד (Commenting)](#solution1)
   - [פתרון 2: Eager Mode](#solution2)
   - [פתרון 3: שימוש ב-pipe()](#solution3)
3. [בדיקה ואופטימיזציה של Query Plan](#query-plan)
   - [הצגה גרפית עם show_graph()](#show-graph)
   - [הצגה טקסטואלית עם explain()](#explain)
   - [אופטימיזציות אוטומטיות](#optimizations)
4. [בדיקת איכות נתונים עם cuallee](#cuallee)
   - [בדיקות שלמות (Completeness)](#completeness)
   - [בדיקות ערכים מקובלים](#accepted-values)
   - [בדיקות סטטיסטיות](#statistical)
   - [שימוש ב-assertions](#assertions)
5. [תרגילים מעשיים](#exercises)
6. [משאבים נוספים](#resources)

---

<a id='setup'></a>
## 1. 🚀 הגדרות ראשוניות

### ייבוא ספריות

ראשית, נייבא את הספריות הדרושות לעבודתנו:

In [1]:
# ייבוא Polars - ספריית DataFrame מהירה ויעילה
import polars as pl

print(f"✅ Polars version: {pl.__version__}")

✅ Polars version: 1.35.1


### טעינת הנתונים

נעבוד עם מערך נתונים של פוקימונים. נטען אותו במצב **Lazy** - כלומר, הפעולות לא יבוצעו מיד אלא רק כשנקרא ל-`collect()`.

In [2]:
# טעינת הנתונים במצב Lazy
# במצב זה, Polars רק "מתכנן" את הפעולות ללא ביצוען
lf = pl.scan_csv('../data/pokemon.csv')

# נציג את 5 השורות הראשונות כדי להבין את המבנה
lf.head().collect()

#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,bool
1,"""Bulbasaur""","""Grass""","""Poison""",318,45,49,49,65,65,45,1,false
2,"""Ivysaur""","""Grass""","""Poison""",405,60,62,63,80,80,60,1,false
3,"""Venusaur""","""Grass""","""Poison""",525,80,82,83,100,100,80,1,false
3,"""VenusaurMega Venusaur""","""Grass""","""Poison""",625,80,100,123,122,120,80,1,false
4,"""Charmander""","""Fire""",null,309,39,52,43,60,50,65,1,false


### 💡 טיפ חשוב: Lazy vs Eager

| מאפיין | Lazy Mode (`LazyFrame`) | Eager Mode (`DataFrame`) |
|---------|------------------------|-------------------------|
| **ביצוע** | נדחה עד ל-`collect()` | מיידי |
| **אופטימיזציה** | אוטומטית | אין |
| **זיכרון** | יעיל יותר | עשוי להיות כבד |
| **מתי להשתמש** | קבצים גדולים, שרשראות ארוכות | חקירת נתונים, פעולות קצרות |

---

<a id='debugging-chained'></a>
## 2. 🔍 ניפוי שגיאות בפעולות משורשרות

<a id='problem'></a>
### הבעיה: שרשראות ארוכות

כאשר אנו כותבים קוד ב-Polars בסגנון של שרשור פעולות (method chaining), קשה לדעת היכן בדיוק אירעה השגיאה.

#### ✅ דוגמה עובדת

הקוד הבא מוסיף דירוגים לפוקימונים לפי ההתקפה, ההגנה והמהירות:

In [3]:
# שרשרת פעולות עובדת
(
    lf
    # הוספת עמודות דירוג עבור כל סטטיסטיקה
    .with_columns(
        pl.col('Attack').rank(method='dense').alias('Atk Rank'),
        pl.col('Defense').rank(method='dense').alias('Def Rank'),
        pl.col('Speed').rank(method='dense').alias('Spe Rank'),
    )
    # בחירת עמודות רלוונטיות בלבד
    .select(
        'Name',           # שם הפוקימון
        'Total',          # סך הסטטיסטיקות
        'Attack',         # התקפה
        'Defense',        # הגנה
        'Speed',          # מהירות
        pl.col('^*Rank$') # כל העמודות המסתיימות ב-Rank (regex)
    )
    # מיון לפי סך הסטטיסטיקות
    .sort('Total')
    # לקיחת 5 הראשונים
    .head()
    # ביצוע השאילתה
    .collect()
)

Name,Total,Attack,Defense,Speed,Atk Rank,Def Rank,Spe Rank
str,i64,i64,i64,i64,u32,u32,u32
"""Sunkern""",180,30,30,30,11,8,11
"""Azurill""",190,20,40,20,4,16,4
"""Kricketot""",194,25,41,25,8,17,8
"""Weedle""",195,35,30,50,13,8,30
"""Wurmple""",195,45,35,20,21,12,4


#### ❌ דוגמה עם שגיאה

מה קורה כאשר יש שגיאת הקלדה?

In [4]:
# ⚠️ קוד זה יזרוק שגיאה!
# שימו לב: 'Deffense' במקום 'Defense'
try:
    (
        lf
        .with_columns(
            pl.col('Attack').rank(method='dense').alias('Atk Rank'),
            pl.col('Defense').rank(method='dense').alias('Def Rank'),
            pl.col('Speed').rank(method='dense').alias('Spe Rank'),
        )
        .select(
            'Name',
            'Total',
            'Attack',
            'Deffense',  # ❌ שגיאת הקלדה!
            'Speed',
            pl.col('^*Rank$')
        )
        .sort('Total')
        .head()
        .collect()
    )
except Exception as e:
    print(f"❌ שגיאה: {type(e).__name__}")
    print(f"📝 הודעה: {str(e)}")

❌ שגיאה: ColumnNotFoundError
📝 הודעה: unable to find column "Deffense"; valid columns: ["#", "Name", "Type 1", "Type 2", "Total", "HP", "Attack", "Defense", "Sp. Atk", "Sp. Def", "Speed", "Generation", "Legendary", "Atk Rank", "Def Rank", "Spe Rank"]

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sink' <---
 WITH_COLUMNS:
 [col("Attack").rank().alias("Atk Rank"), col("Defense").rank().alias("Def Rank"), col("Speed").rank().alias("Spe Rank")] 
  Csv SCAN [../data/pokemon.csv]
  PROJECT */13 COLUMNS
  ESTIMATED ROWS: 808


### 🎯 איך לאתר את הבעיה?

הודעת השגיאה אומרת שהעמודה לא קיימת, אבל **לא ברור באיזה שלב** בשרשרת הפעולות!

<a id='solution1'></a>
### ✅ פתרון 1: הערת קוד (Commenting Out)

הגישה הפשוטה ביותר: להעיר שורות בשרשרת כדי לבדוק איפה השגיאה.

In [5]:
# נעיר את הפעולה select() כדי לראות את התוצאה הביניימית
(
    lf
    .with_columns(
        pl.col('Attack').rank(method='dense').alias('Atk Rank'),
        pl.col('Defense').rank(method='dense').alias('Def Rank'),
        pl.col('Speed').rank(method='dense').alias('Spe Rank'),
    )
    # .select(  # ← העירו את זה
    #     'Name',
    #     'Total',
    #     'Attack',
    #     'Deffense',
    #     'Speed',
    #     pl.col('^*Rank$')
    # )
    .sort('Total')
    .head()
    .collect()
)

#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Atk Rank,Def Rank,Spe Rank
i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,bool,u32,u32,u32
191,"""Sunkern""","""Grass""",null,180,30,30,30,30,30,30,2,false,11,8,11
298,"""Azurill""","""Normal""","""Fairy""",190,50,20,40,20,40,20,3,false,4,16,4
401,"""Kricketot""","""Bug""",null,194,37,25,41,25,41,25,4,false,8,17,8
13,"""Weedle""","""Bug""","""Poison""",195,40,35,30,20,20,50,1,false,13,8,30
265,"""Wurmple""","""Bug""",null,195,45,45,35,20,30,20,3,false,21,12,4


#### 💭 מה למדנו?

כאשר מסירים את ה-`select()`, הקוד עובד! זה אומר שהבעיה היא **בשלב של select**.

עכשיו אפשר לבדוק את רשימת העמודות בשלב select ולגלות את השגיאה: `'Deffense'` במקום `'Defense'`.

<a id='solution2'></a>
### ✅ פתרון 2: Eager Mode

גישה נוספת: להריץ את הקוד ב-**Eager mode** על ידי הוספת `collect()` מוקדם יותר.

In [6]:
# ⚠️ קוד זה יזרוק שגיאה - אבל בשלב מוקדם יותר!
try:
    (
        lf
        .collect()  # ← collect() בהתחלה הופך את זה ל-DataFrame
        .with_columns(
            pl.col('Attack').rank(method='dense').alias('Atk Rank'),
            pl.col('Defense').rank(method='dense').alias('Def Rank'),
            pl.col('Speed').rank(method='dense').alias('Spe Rank'),
        )
        .select(
            'Name',
            'Total',
            'Attack',
            'Deffense',  # ❌ השגיאה תתגלה כאן!
            'Speed',
            pl.col('^*Rank$')
        )
        .sort('Total')
        .head()
    )
except Exception as e:
    print(f"❌ שגיאה בשלב: .select()")
    print(f"📝 הודעה: {str(e)}")

❌ שגיאה בשלב: .select()
📝 הודעה: unable to find column "Deffense"; valid columns: ["#", "Name", "Type 1", "Type 2", "Total", "HP", "Attack", "Defense", "Sp. Atk", "Sp. Def", "Speed", "Generation", "Legendary", "Atk Rank", "Def Rank", "Spe Rank"]


#### 🤔 מתי להשתמש ב-Eager Mode?

**יתרונות:**
- השגיאה מופיעה בדיוק באותו שלב שבו היא מתרחשת
- קל יותר לדבג

**חסרונות:**
- איטי יותר (אין אופטימיזציה)
- צורך יותר זיכרון

💡 **המלצה:** השתמשו ב-Eager Mode רק בזמן debug, ואז חזרו ל-Lazy Mode.

<a id='solution3'></a>
### ✅ פתרון 3: שימוש ב-pipe() - הפתרון המומלץ!

הפתרון המקצועי ביותר: **לפצל את השרשרת לפונקציות** ולהשתמש ב-`pipe()`.

#### 🏗️ בניית פונקציות מודולריות

נפצל את השרשרת לפונקציות קטנות ונבדקות:

In [7]:
def add_ranks(lf: pl.LazyFrame) -> pl.LazyFrame:
    """
    מוסיף עמודות דירוג לכל סטטיסטיקה
    
    Parameters:
    -----------
    lf : pl.LazyFrame
        טבלה עם עמודות Attack, Defense, Speed
        
    Returns:
    --------
    pl.LazyFrame
        טבלה עם 3 עמודות דירוג נוספות
    """
    return (
        lf
        .with_columns(
            pl.col('Attack').rank(method='dense').alias('Atk Rank'),
            pl.col('Defense').rank(method='dense').alias('Def Rank'),
            pl.col('Speed').rank(method='dense').alias('Spe Rank'),
        )
    )

def keep_cols(lf: pl.LazyFrame) -> pl.LazyFrame:
    """
    בוחר רק את העמודות הרלוונטיות
    
    Parameters:
    -----------
    lf : pl.LazyFrame
        טבלה מלאה
        
    Returns:
    --------
    pl.LazyFrame
        טבלה מסוננת עם עמודות נבחרות בלבד
    """
    return (
        lf
        .select(
            'Name',
            'Total',
            'Attack',
            'Defense',  # ✅ כאן תיקנו את השגיאה!
            'Speed',
            pl.col('^*Rank$')  # בחירת כל העמודות שמסתיימות ב-Rank
        )
    )

# בדיקת הפונקציה add_ranks בנפרד
print("✅ בדיקת add_ranks:")
lf.pipe(add_ranks).head().collect()

✅ בדיקת add_ranks:


#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Atk Rank,Def Rank,Spe Rank
i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,bool,u32,u32,u32
1,"""Bulbasaur""","""Grass""","""Poison""",318,45,49,49,65,65,45,1,false,25,24,25
2,"""Ivysaur""","""Grass""","""Poison""",405,60,62,63,80,80,60,1,false,38,38,39
3,"""Venusaur""","""Grass""","""Poison""",525,80,82,83,100,100,80,1,false,58,56,59
3,"""VenusaurMega Venusaur""","""Grass""","""Poison""",625,80,100,123,122,120,80,1,false,74,87,59
4,"""Charmander""","""Fire""",null,309,39,52,43,60,50,65,1,false,28,19,44


In [8]:
# בדיקת הפונקציה keep_cols בנפרד
print("✅ בדיקת keep_cols:")
lf.pipe(add_ranks).pipe(keep_cols).head().collect()

✅ בדיקת keep_cols:


Name,Total,Attack,Defense,Speed,Atk Rank,Def Rank,Spe Rank
str,i64,i64,i64,i64,u32,u32,u32
"""Bulbasaur""",318,49,49,45,25,24,25
"""Ivysaur""",405,62,63,60,38,38,39
"""Venusaur""",525,82,83,80,58,56,59
"""VenusaurMega Venusaur""",625,100,123,80,74,87,59
"""Charmander""",309,52,43,65,28,19,44


#### 🔗 שימוש ב-pipe()

עכשיו נוכל לשרשר את הפונקציות בצורה נקייה וברורה:

In [9]:
# שרשרת מודולרית עם pipe()
result = (
    lf
    .pipe(add_ranks)      # שלב 1: הוספת דירוגים
    .pipe(keep_cols)      # שלב 2: סינון עמודות
    .sort('Total')        # שלב 3: מיון
    .head()               # שלב 4: לקיחת 5 הראשונים
    .collect()            # שלב 5: ביצוע
)

result

Name,Total,Attack,Defense,Speed,Atk Rank,Def Rank,Spe Rank
str,i64,i64,i64,i64,u32,u32,u32
"""Sunkern""",180,30,30,30,11,8,11
"""Azurill""",190,20,40,20,4,16,4
"""Kricketot""",194,25,41,25,8,17,8
"""Weedle""",195,35,30,50,13,8,30
"""Wurmple""",195,45,35,20,21,12,4


### 🌟 יתרונות השימוש ב-pipe()

| יתרון | הסבר |
|-------|------|
| **קריאות** | כל פונקציה עושה דבר אחד ברור |
| **בדיקות** | אפשר לבדוק כל פונקציה בנפרד |
| **שימוש חוזר** | אפשר להשתמש באותן פונקציות במקומות שונים |
| **דיבאג קל** | קל לזהות היכן הבעיה |
| **תיעוד** | אפשר להוסיף docstrings לכל פונקציה |

---

<a id='query-plan'></a>
## 3. 📊 בדיקה ואופטימיזציה של Query Plan

### מה זה Query Plan?

**Query Plan** הוא "תוכנית הביצוע" של Polars - כלומר, האסטרטגיה שבה Polars מתכנן לבצע את השאילתה.

Polars מבצע **אופטימיזציות אוטומטיות** כמו:
- **Predicate Pushdown** - העברת סינונים קדימה
- **Projection Pushdown** - קריאת רק העמודות הנחוצות
- **Query Coalescing** - איחוד פעולות דומות

### הכנת דוגמה

נוסיף פונקציה נוספת לשרשרת שלנו:

In [10]:
def keep_grass_or_fire(lf: pl.LazyFrame) -> pl.LazyFrame:
    """
    מסנן רק פוקימונים מסוג דשא או אש
    
    Parameters:
    -----------
    lf : pl.LazyFrame
        טבלה עם עמודות Type 1 ו-Type 2
        
    Returns:
    --------
    pl.LazyFrame
        טבלה מסוננת
    """
    accepted_types = ['Grass', 'Fire']
    return (
        lf
        .filter(
            # תנאי OR: אם Type 1 או Type 2 הם דשא או אש
            (pl.col('Type 1').is_in(accepted_types))
            | (pl.col('Type 2').is_in(accepted_types))
        )
    )

<a id='show-graph'></a>
### 📈 הצגה גרפית עם show_graph()

המתודה `show_graph()` מציגה את תוכנית השאילתה בצורה ויזואלית.

In [11]:
# תוכנית שאילתה מאופטמת (ברירת מחדל)
(
    lf
    .pipe(add_ranks)
    .pipe(keep_grass_or_fire)
    .pipe(keep_cols)
    .show_graph()
)

ImportError: the graphviz `dot` binary should be on your PATH.(If not installed you can download here: https://graphviz.org/download/)

#### 🔍 מה רואים בגרף?

- **צמתים (Nodes):** כל פעולה בשאילתה
- **חיצים (Edges):** זרימת הנתונים
- **צבעים:** סוגי פעולות שונים

💡 **שימו לב:** Polars שינה את סדר הפעולות לביצועים מיטביים!

In [12]:
# תוכנית שאילתה ללא אופטימיזציה
(
    lf
    .pipe(add_ranks)
    .pipe(keep_grass_or_fire)
    .pipe(keep_cols)
    .show_graph(optimized=False)  # ללא אופטימיזציה!
)

ImportError: the graphviz `dot` binary should be on your PATH.(If not installed you can download here: https://graphviz.org/download/)

#### 🆚 השוואה: מאופטם vs לא מאופטם

| היבט | לא מאופטם | מאופטם |
|------|-----------|--------|
| **סדר הפעולות** | לפי הקוד | משונה לפי יעילות |
| **קריאת קובץ** | כל העמודות | רק העמודות הנחוצות |
| **סינונים** | בסוף | בהתחלה |
| **ביצועים** | איטי | מהיר |

---

<a id='explain'></a>
### 📝 הצגה טקסטואלית עם explain()

המתודה `explain()` מציגה את תוכנית השאילתה בפורמט טקסט.

In [13]:
# תוכנית מאופטמת (ברירת מחדל)
print(
    lf
    .pipe(add_ranks)
    .pipe(keep_grass_or_fire)
    .pipe(keep_cols)
    .explain()
)

simple π 8/8 ["Name", "Total", "Attack", ... 5 other columns]
  FILTER [(col("Type 1").is_in([["Grass", "Fire"]])) | (col("Type 2").is_in([["Grass", "Fire"]]))]
  FROM
     WITH_COLUMNS:
     [col("Attack").rank().alias("Atk Rank"), col("Defense").rank().alias("Def Rank"), col("Speed").rank().alias("Spe Rank")] 
      Csv SCAN [../data/pokemon.csv]
      PROJECT 7/13 COLUMNS
      ESTIMATED ROWS: 808


#### 🔎 כיצד לקרוא את ה-explain?

תוכנית השאילתה נקראת **מלמטה למעלה**:

1. **Csv SCAN** - קריאת קובץ CSV
   - שימו לב: רק עמודות נחוצות נקראות!
2. **FILTER** - סינון הנתונים
3. **WITH_COLUMNS** - הוספת עמודות חדשות
4. **SELECT** - בחירת עמודות

💡 **הבחינו:** הסינון (FILTER) הועבר **לפני** הוספת העמודות!

In [14]:
# תוכנית ללא אופטימיזציה
print(
    lf
    .pipe(add_ranks)
    .pipe(keep_grass_or_fire)
    .pipe(keep_cols)
    .explain(optimized=False)
)

SELECT [col("Name"), col("Total"), col("Attack"), col("Defense"), col("Speed"), col("Atk Rank"), col("Def Rank"), col("Spe Rank")]
  FILTER [(col("Type 1").is_in([["Grass", "Fire"]])) | (col("Type 2").is_in([["Grass", "Fire"]]))]
  FROM
     WITH_COLUMNS:
     [col("Attack").rank().alias("Atk Rank"), col("Defense").rank().alias("Def Rank"), col("Speed").rank().alias("Spe Rank")] 
      Csv SCAN [../data/pokemon.csv]
      PROJECT */13 COLUMNS
      ESTIMATED ROWS: 808


#### 📊 ההבדל בין מאופטם ללא מאופטם

**ללא אופטימיזציה:**
- הפעולות מתבצעות בדיוק לפי סדר הקוד
- כל העמודות נקראות מהקובץ
- פחות יעיל

**עם אופטימיזציה:**
- Polars משנה את הסדר
- רק עמודות נחוצות נקראות
- יעיל בהרבה!

---

<a id='optimizations'></a>
### ⚡ אופטימיזציות אוטומטיות

#### דוגמה: סדר הפעולות חשוב!

נשווה בין שני סדרי פעולות:

In [15]:
# סדר 1: add_ranks → filter → select
print("=== סדר 1: הוספת דירוגים לפני סינון ===")
print(
    lf
    .pipe(add_ranks)           # 1. הוספת דירוגים לכל השורות
    .pipe(keep_grass_or_fire)  # 2. סינון
    .pipe(keep_cols)           # 3. בחירת עמודות
    .explain()
)

print("\n" + "="*50 + "\n")

# סדר 2: filter → select → add_ranks
print("=== סדר 2: סינון לפני הוספת דירוגים ===")
print(
    lf
    .pipe(keep_grass_or_fire)  # 1. סינון
    .pipe(keep_cols)           # 2. בחירת עמודות  
    .pipe(add_ranks)           # 3. הוספת דירוגים רק לשורות מסוננות
    .explain()
)

=== סדר 1: הוספת דירוגים לפני סינון ===
simple π 8/8 ["Name", "Total", "Attack", ... 5 other columns]
  FILTER [(col("Type 1").is_in([["Grass", "Fire"]])) | (col("Type 2").is_in([["Grass", "Fire"]]))]
  FROM
     WITH_COLUMNS:
     [col("Attack").rank().alias("Atk Rank"), col("Defense").rank().alias("Def Rank"), col("Speed").rank().alias("Spe Rank")] 
      Csv SCAN [../data/pokemon.csv]
      PROJECT 7/13 COLUMNS
      ESTIMATED ROWS: 808


=== סדר 2: סינון לפני הוספת דירוגים ===
 WITH_COLUMNS:
 [col("Attack").rank().alias("Atk Rank"), col("Defense").rank().alias("Def Rank"), col("Speed").rank().alias("Spe Rank")] 
  simple π 5/5 ["Name", "Total", "Attack", ... 2 other columns]
    Csv SCAN [../data/pokemon.csv]
    PROJECT 7/13 COLUMNS
    SELECTION: [(col("Type 1").is_in([["Grass", "Fire"]])) | (col("Type 2").is_in([["Grass", "Fire"]]))]
    ESTIMATED ROWS: 808


#### 🎯 מה למדנו?

**סדר 2 יעיל יותר כי:**
1. מסנן את הנתונים מוקדם - פחות שורות לעבד
2. מוסיף דירוגים רק לשורות רלוונטיות
3. חוסך זמן וזיכרון

💡 **כלל אצבע:** מסנן ובחר עמודות מוקדם ככל האפשר!

---

### 🌊 Streaming Mode

עבור קבצים גדולים מאוד, Polars תומך ב-**Streaming Mode** - עיבוד הנתונים בחלקים.

In [16]:
# הצגת תוכנית עם streaming
print(
    lf
    .pipe(keep_grass_or_fire)
    .pipe(keep_cols)    
    .pipe(add_ranks)
    .explain(streaming=True)
)

 WITH_COLUMNS:
 [col("Attack").rank().alias("Atk Rank"), col("Defense").rank().alias("Def Rank"), col("Speed").rank().alias("Spe Rank")] 
  simple π 5/5 ["Name", "Total", "Attack", ... 2 other columns]
    Csv SCAN [../data/pokemon.csv]
    PROJECT 7/13 COLUMNS
    SELECTION: [(col("Type 1").is_in([["Grass", "Fire"]])) | (col("Type 2").is_in([["Grass", "Fire"]]))]
    ESTIMATED ROWS: 808


/var/folders/kb/bm6sw7g57_5_708m47q7jqzm0000gn/T/ipykernel_5664/2653982378.py:7: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  .explain(streaming=True)


#### 🌊 מתי להשתמש ב-Streaming?

| מצב | האם streaming מתאים? |
|-----|---------------------|
| הקובץ גדול מהזיכרון | ✅ כן |
| צריך לעבד מיליוני שורות | ✅ כן |
| השאילתה פשוטה (filter, select) | ✅ כן |
| צריך join מורכב | ❌ לא תמיד |
| צריך window functions | ❌ לא תמיד |

---

<a id='cuallee'></a>
## 4. ✅ בדיקת איכות נתונים עם cuallee

### מה זה cuallee?

**cuallee** היא ספרייה לבדיקת איכות נתונים (Data Quality). היא מאפשרת:
- בדיקת שלמות נתונים
- בדיקת ייחודיות
- בדיקת ערכים מקובלים
- בדיקות סטטיסטיות

### התקנה

```bash
pip install cuallee
```

In [17]:
# ייבוא cuallee
from cuallee import Check, CheckLevel

# המרת LazyFrame ל-DataFrame (cuallee עובד עם DataFrame)
df = lf.collect()
df.head()

#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,bool
1,"""Bulbasaur""","""Grass""","""Poison""",318,45,49,49,65,65,45,1,false
2,"""Ivysaur""","""Grass""","""Poison""",405,60,62,63,80,80,60,1,false
3,"""Venusaur""","""Grass""","""Poison""",525,80,82,83,100,100,80,1,false
3,"""VenusaurMega Venusaur""","""Grass""","""Poison""",625,80,100,123,122,120,80,1,false
4,"""Charmander""","""Fire""",null,309,39,52,43,60,50,65,1,false


<a id='completeness'></a>
### 🎯 בדיקות שלמות (Completeness)

בדיקת האם אין ערכים חסרים (null) בעמודות.

In [19]:
# יצירת אובייקט בדיקה
check = Check(CheckLevel.WARNING, 'Completeness')

# ביצוע בדיקות
result = (
    check
    .is_complete('Name')    # האם כל השורות יש להן שם?
    .is_unique('Name')      # האם כל השמות ייחודיים?
    .validate(df)           # הרצת הבדיקות
)

result

Dependency modules missing: No module named 'pandas'


id,timestamp,check,level,column,rule,value,rows,violations,pass_rate,pass_threshold,status
i64,str,str,str,str,str,str,i64,i64,f64,f64,str
1,"""2025-11-03 08:12:22""","""Completeness""","""WARNING""","""Name""","""is_complete""","""N/A""",800,0,1.0,1.0,"""PASS"""
2,"""2025-11-03 08:12:22""","""Completeness""","""WARNING""","""Name""","""is_unique""","""N/A""",800,0,1.0,1.0,"""PASS"""


#### 📊 הבנת התוצאות

| עמודה | משמעות |
|--------|--------|
| **check** | שם חבילת הבדיקות |
| **column** | העמודה שנבדקה |
| **rule** | הכלל שנבדק |
| **rows** | מספר השורות |
| **violations** | מספר ההפרות |
| **pass_rate** | אחוז הצלחה |
| **status** | PASS או FAIL |

✅ **PASS** = הבדיקה עברה בהצלחה  
❌ **FAIL** = יש בעיה בנתונים

<a id='accepted-values'></a>
### 🎨 בדיקות ערכים מקובלים

בדיקה שהערכים בעמודה שייכים לרשימה מוגדרת מראש.

In [20]:
# שלב 1: חילוץ רשימת הערכים המקובלים
accepted_types = (
    lf.select('Type 1')
    .unique()              # ערכים ייחודיים בלבד
    .collect()
    .to_series()           # המרה ל-Series
    .to_list()             # המרה לרשימת Python
)

print(f"סוגים מקובלים: {accepted_types}")
print(f"סה\"כ: {len(accepted_types)} סוגים")

סוגים מקובלים: ['Flying', 'Ice', 'Normal', 'Fighting', 'Dark', 'Water', 'Ghost', 'Electric', 'Rock', 'Fire', 'Steel', 'Ground', 'Fairy', 'Dragon', 'Grass', 'Poison', 'Bug', 'Psychic']
סה"כ: 18 סוגים


In [21]:
# שלב 2: בדיקה שכל הערכים תקינים
check = Check(CheckLevel.WARNING, 'Accepted Values')

(
    check
    .is_contained_in('Type 1', set(accepted_types))  # האם כל Type 1 ברשימה?
    .validate(df)
    .select('check', 'column', 'rule', 'status')
)

Dependency modules missing: No module named 'pandas'


check,column,rule,status
str,str,str,str
"""Accepted Values""","""Type 1""","""is_contained_in""","""PASS"""


#### 💡 למה זה שימושי?

דמיינו שיש שגיאת הקלדה בנתונים:
- `"Grass"` (נכון) vs `"Grss"` (שגיאה)
- `"Fire"` (נכון) vs `"Frie"` (שגיאה)

הבדיקה הזו תתפוס את השגיאות האלו!

---

<a id='statistical'></a>
### 📈 בדיקות סטטיסטיות

בדיקת תנאים נומריים על עמודות.

In [22]:
# רשימת עמודות הסטטיסטיקות
stats_cols = [
    'HP',        # נקודות חיים
    'Attack',    # התקפה
    'Defense',   # הגנה
    'Sp. Atk',   # התקפה מיוחדת
    'Sp. Def',   # הגנה מיוחדת
    'Speed'      # מהירות
]

# עמודות לתוצאה
res_cols = [
    'check',       # שם הבדיקה
    'column',      # עמודה
    'rule',        # כלל
    'rows',        # מספר שורות
    'violations',  # מספר הפרות
    'pass_rate',   # אחוז הצלחה
    'status'       # סטטוס
]

In [23]:
# ביצוע בדיקות
check = Check(CheckLevel.WARNING, 'Validation on Stats')

result = (
    check
    .is_complete(stats_cols)        # כל הערכים קיימים?
    .is_greater_than(stats_cols, 0) # כל הערכים חיוביים?
    .validate(df)
    .select(res_cols)
)

result

Dependency modules missing: No module named 'pandas'


check,column,rule,rows,violations,pass_rate,status
str,str,str,i64,i64,f64,str
"""Validation on Stats""","""('HP', 'Attack', 'Defense', 'S…","""is_complete""",800,0,1.0,"""PASS"""
"""Validation on Stats""","""('HP', 'Attack', 'Defense', 'S…","""is_greater_than""",800,0,1.0,"""PASS"""


#### 🔍 ניתוח התוצאות

מהטבלה למעלה נוכל לראות:
- האם יש ערכים חסרים בסטטיסטיקות?
- האם יש ערכים שליליים או אפס?
- מה אחוז ההצלחה של כל בדיקה?

### 🔄 בדיקות על מספר עמודות בבת אחת

cuallee מאפשר לבדוק מספר עמודות בפקודה אחת.

In [24]:
check = Check(CheckLevel.WARNING, 'Multiple Columns Check')
cols = ['Name', 'Type 1', 'Type 2']

result = (
    check
    .are_complete(cols)  # are_complete במקום is_complete
    .are_unique(cols)    # are_unique במקום is_unique
    .validate(df)
    .select(res_cols)
)

result

Dependency modules missing: No module named 'pandas'


check,column,rule,rows,violations,pass_rate,status
str,str,str,i64,f64,f64,str
"""Multiple Columns Check""","""('Name', 'Type 1', 'Type 2')""","""are_complete""",800,128.666667,0.839167,"""FAIL"""
"""Multiple Columns Check""","""('Name', 'Type 1', 'Type 2')""","""are_unique""",800,0.0,1.0,"""PASS"""


#### 📝 הבחנה חשובה

| פקודה | שימוש |
|-------|-------|
| `is_complete('Name')` | עבור עמודה **אחת** |
| `are_complete(['Name', 'Type 1'])` | עבור **מספר** עמודות |
| `is_unique('Name')` | עבור עמודה **אחת** |
| `are_unique(['Name', 'Type 1'])` | עבור **מספר** עמודות |

---

<a id='assertions'></a>
### ⚠️ שימוש ב-assertions

אפשר להמיר את תוצאות cuallee ל-**assertions** שיעצרו את הקוד אם הבדיקה נכשלת.

In [25]:
# בדיקה שתיכשל
check = Check(CheckLevel.WARNING, 'Completeness')

try:
    result = (
        check
        .is_complete('Type 2')  # Type 2 יש בו ערכים ריקים!
        .validate(df)
        .select('status')[0,0] == 'PASS'
    )
    assert result, "❌ הבדיקה נכשלה: Type 2 לא שלם"
    print("✅ הבדיקה עברה בהצלחה")
except AssertionError as e:
    print(f"⚠️ {e}")

Dependency modules missing: No module named 'pandas'


⚠️ ❌ הבדיקה נכשלה: Type 2 לא שלם


#### 🎯 מתי להשתמש ב-assertions?

**תרחישים מומלצים:**
- בתחילת pipeline - וידוא שהנתונים תקינים
- בסוף pipeline - וידוא שהתוצאות נכונות
- ב-CI/CD - עצירת deployment אם הנתונים לא תקינים
- בבדיקות אוטומטיות (pytest)

---

<a id='exercises'></a>
## 5. 💪 תרגילים מעשיים

### תרגיל 1: ניפוי שגיאות

הקוד הבא מכיל שגיאה. מצאו ותקנו אותה:

In [ ]:
# ⚠️ קוד עם שגיאה - תקנו אותה!
# רמז: השגיאה היא בשם עמודה

# try:
#     result = (
#         lf
#         .filter(pl.col('Total') > 500)
#         .select(
#             'Name',
#             'Type1',  # ← יש כאן שגיאה!
#             'Total'
#         )
#         .head()
#         .collect()
#     )
# except Exception as e:
#     print(f"שגיאה: {e}")

# כתבו כאן את הקוד המתוקן:


### תרגיל 2: יצירת פונקציות עם pipe

צרו פונקציה שמסננת פוקימונים חזקים (Total > 500) ומחזירה רק את השם והסטטיסטיקות:

In [ ]:
# כתבו כאן את הפונקציה שלכם
def filter_strong_pokemon(lf: pl.LazyFrame) -> pl.LazyFrame:
    """
    מסנן פוקימונים עם Total > 500
    ומחזיר רק עמודות רלוונטיות
    """
    # הקוד שלכם כאן
    pass

# בדיקה:
# lf.pipe(filter_strong_pokemon).head().collect()

### תרגיל 3: אופטימיזציה

השוו בין שני הסדרים הבאים והסבירו איזה יעיל יותר:

In [ ]:
# סדר A
# print("סדר A:")
# print(
#     lf
#     .with_columns(pl.col('Total') * 2)
#     .filter(pl.col('Total') > 500)
#     .select('Name', 'Total')
#     .explain()
# )

# סדר B
# print("\nסדר B:")
# print(
#     lf
#     .filter(pl.col('Total') > 500)
#     .select('Name', 'Total')
#     .with_columns(pl.col('Total') * 2)
#     .explain()
# )

# איזה סדר יעיל יותר ולמה?

### תרגיל 4: בדיקות איכות

צרו בדיקת cuallee שבודקת:
1. העמודה 'Generation' מכילה רק מספרים 1-6
2. העמודה 'Legendary' מכילה רק True/False

In [ ]:
# כתבו כאן את הבדיקות שלכם
# check = Check(CheckLevel.WARNING, 'Exercise 4')
# ...

### תרגיל 5: אתגר מתקדם

צרו pipeline מלא שמבצע:
1. טעינת הנתונים
2. סינון פוקימונים מסוג Water או Electric
3. הוספת עמודה חדשה: `power_ratio = Attack / Defense`
4. מיון לפי `power_ratio`
5. בדיקת cuallee שכל הערכים חיוביים
6. החזרת 10 הראשונים

In [ ]:
# כתבו כאן את הפתרון שלכם
# השתמשו בפונקציות עם pipe()


---

## 📚 סיכום הפרק

### מה למדנו?

✅ **ניפוי שגיאות:**
- שלוש שיטות: commenting, eager mode, pipe()
- pipe() היא השיטה המומלצת

✅ **Query Plan:**
- `show_graph()` להצגה ויזואלית
- `explain()` להצגה טקסטואלית
- Polars מבצע אופטימיזציות אוטומטיות

✅ **איכות נתונים:**
- cuallee לבדיקות איכות
- בדיקות שלמות, ייחודיות, ערכים מקובלים
- שימוש ב-assertions

### טיפים לזכירה

| טיפ | הסבר |
|-----|------|
| 🔍 | השתמשו ב-pipe() לקוד מודולרי |
| ⚡ | סננו ובחרו עמודות מוקדם |
| 📊 | השתמשו ב-explain() להבנת האופטימיזציות |
| ✅ | הוסיפו בדיקות cuallee לפני ואחרי pipeline |
| 🌊 | השתמשו ב-streaming לקבצים גדולים |

---

<a id='resources'></a>
## 🔗 משאבים נוספים

### תיעוד רשמי
- [Polars Documentation](https://pola-rs.github.io/polars/)
- [Polars User Guide](https://pola-rs.github.io/polars-book/)
- [cuallee Documentation](https://canimus.github.io/cuallee/)

### מאמרים מומלצים
- [Debugging Polars Queries](https://www.rhosignal.com/posts/polars-debugging/)
- [Query Optimization in Polars](https://www.pola.rs/posts/polars_fast_dataframes/)
- [Data Quality Testing](https://towardsdatascience.com/data-quality-testing)

### קהילה
- [Polars Discord](https://discord.gg/4UfP5cfBE7)
- [Polars GitHub](https://github.com/pola-rs/polars)
- [Stack Overflow - Polars Tag](https://stackoverflow.com/questions/tagged/python-polars)

### וידאו והדרכות
- [Polars YouTube Channel](https://www.youtube.com/@polars-tech)
- [Data Engineering with Polars](https://www.datacamp.com/courses/data-engineering-with-polars)

---

## 🎉 סיימתם את המדריך!

כל הכבוד! 🎊 עכשיו אתם מצוידים בכלים לכתיבת קוד Polars:
- ✅ נקי ומודולרי
- ✅ מאופטם וביצועי
- ✅ עם בדיקות איכות
- ✅ קל לדיבאג ותחזוקה

**המשך ללמוד ולתרגל!** 🚀